In [6]:
from compas.datastructures import Mesh
from compas.datastructures import subdivision as sd
from compas_plotters import MeshPlotter
from compas.geometry import midpoint_point_point, Vector, distance_point_point, convex_hull
from shapes import Sphere, Cylinder
import random
from statistics import mean


from noise import pnoise1, snoise2, pnoise2, perlin
from utilities import draw_compas_mesh, export_obj_by_attribute
import mysubdivision as msd


def plot(m):
    plotter = MeshPlotter(m)
    plotter.draw_edges()
    plotter.draw_faces()
    plotter.show()

mesh = Mesh.from_json('ring_1.json')

draw_compas_mesh(mesh,size=35

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [2]:
def noises(bigY, bigX, octaves=1):
    freq = 16.0 * octaves

    noise_list = []

    for y in range(bigY):
        noise_list.append(pnoise2(-y / freq, y / freq, octaves))
    
    return noise_list

In [3]:
# Start subdividing

# weird setup
inner_faces = []
for inner_face in mesh.faces_where({'location': 'inner'}):
    inner_faces.append(inner_face)
    
bigY = len(inner_faces)

noise_values = noises(bigY, 3)
min_noise = min(noise_values)
max_noise = max(noise_values)

print(noise_values[0:100])
    

[0.0, -0.06250430643558502, -0.125193253159523, -0.18898651003837585, -0.2553577423095703, -0.3246508240699768, -0.39393478631973267, -0.45593464374542236, -0.5, -0.515072226524353, -0.49366897344589233, -0.43535369634628296, -0.3481578826904297, -0.24697476625442505, -0.14888513088226318, -0.06637763977050781, 0.0, 0.059866756200790405, 0.10302478820085526, 0.11263629794120789, 0.07688426971435547, -0.005884647369384766, -0.12460941076278687, -0.2569998502731323, -0.375, -0.4526512622833252, -0.473795622587204, -0.436926931142807, -0.3552846908569336, -0.2518080472946167, -0.15040850639343262, -0.06650674343109131, 0.0, 0.06416430324316025, 0.133090540766716, 0.20058415830135345, 0.2553577423095703, 0.28774985671043396, 0.2942006289958954, 0.2785218358039856, 0.25, 0.21938425302505493, 0.19446644186973572, 0.17704695463180542, 0.16255760192871094, 0.14259594678878784, 0.10939866304397583, 0.06029099225997925, 0.0, -0.06415507197380066, -0.13263961672782898, -0.196719229221344, -0.2392

In [7]:
mesh = Mesh.from_json('ring_1.json')


def move_vertices(mesh, fk, height):
    vertices = mesh.face_vertices(fk)
    vertices_coord = []
    
    face_location = mesh.face_centroid(fk)

    if height < 0:
        direction = Vector.from_start_end([0,0,0], face_location )
    else:
        direction = Vector.from_start_end(face_location, [0,0,0])
    direction.unitize()
    direction = [i * abs(height) * abs(pnoise1(fk, 2)) for i in direction]

    for v in vertices:
        vertices_coord = mesh.vertex_coordinates(v)

        new_loc = Vector.sum_vectors([direction, vertices_coord])

        mesh.vertex[v]['x'] = new_loc[0]
        mesh.vertex[v]['y'] = new_loc[1]
        mesh.vertex[v]['z'] = new_loc[2]
        


def subdivide_by_ftype(mesh):
    fkeys = list(mesh.faces())
    for fk in fkeys:
        location = mesh.get_face_attribute(fk, 'location')
    
        # OUTER RING MAHIRO 
        if location == 'outer':
            
            new_keys = msd.taper_face(mesh, fk, height=random.randint(0,2), ratio=0.05)
            areas = [] 
            for new_key in new_keys:
                ind = new_keys.index(new_key)
                areas.append(mesh.face_area(new_key))
                if mesh.face_area(new_key) > mean(areas):
                    mesh.set_face_attribute(new_key, 'location', 'hub')
                else:
                    mesh.set_face_attribute(new_key, 'location', 'circulation')
        
        elif location == 'circulation':
              # these are side volume
            new_keys = msd.taper_face(mesh, fk, height=1.5, ratio=0.01)
            if random.randint(0, 3) < 1:
                mesh.set_faces_attribute(new_keys, 'location', 'depot')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'conduits')
        
        elif location == 'conduits':
            new_keys = msd.segment_face(mesh, fk, num=3, start_index=0)
            for new_key in new_keys:
                ind = new_keys.index(new_key)
                if ind > 2:
                    mesh.set_face_attribute(new_key, 'location', 'line')
                else:
                    mesh.set_face_attribute(new_key, 'location', 'line2')
                    
                    
        elif location == 'hub':
            new_keys = msd.segment_face(mesh, fk, num=5, start_index=0)
            for new_key in new_keys:
                ind = new_keys.index(new_key)
                if ind == 0:
                    mesh.set_face_attribute(new_key, 'location', 'part_hub_0')
                elif ind == 1:
                    mesh.set_face_attribute(new_key, 'location', 'part_hub_1')
                elif ind == 2:
                    mesh.set_face_attribute(new_key, 'location', 'part_hub_2')
                else:
                    mesh.set_face_attribute(new_key, 'location', 'None')
        
        elif location == 'part_hub_0':
            new_keys = msd.taper_face(mesh, fk, height=1, ratio=0.02)
            mesh.set_faces_attribute(new_keys, 'location', 'part_hub_00')
        elif location == "part_hub_1":
            new_keys = msd.taper_face(mesh, fk, height=1, ratio=0.03)
            mesh.set_faces_attribute(new_keys, 'location', 'part_hub_11')
        elif location == "part_hub_2":
            new_keys = msd.taper_face(mesh, fk, height=1, ratio=0.02)
            mesh.set_faces_attribute(new_keys, 'location', 'part_hub_22')
        
        elif location == 'line':
            pass
        
        elif location == 'line2':
            new_keys = msd.taper_face(mesh, fk, height=1, ratio=0.02)
            for new_key in new_keys:
                ind = new_keys.index(new_key)
                if ind % 2 == 0:
                    mesh.set_face_attribute(new_key, 'location', 'line2_0')
                elif ind % 2 == 1:
                    mesh.set_face_attribute(new_key, 'location', 'line2_1')        
        
        
        elif location == 'inner':
            new_keys = []
            temp_keys = msd.segment_face(mesh, fk, num=4, start_index=0)
            
            for key in temp_keys:
                new_key = msd.segment_face(mesh, key, num=4, start_index=0)
                new_keys.append(new_key)
            
            new_keys += temp_keys
            
            mesh.set_faces_attribute(temp_keys, 'location', 'inner_segmented')
            
        elif location == 'inner_segmented':
            
            noise_value = random.randint(0, bigY)
            
            new_keys = msd.pyramid_face(mesh, fk, height=random.random()+1)
                       
            if fk > noise_value:
                mesh.set_faces_attribute(new_keys, 'location', 'mountain_region')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'sea')
        
        elif location == 'mountain_region':
            move_vertices(mesh, fk, 8)
            
            new_keys = msd.segment_face(mesh, fk, num=4)
            
            mesh.set_face_attribute(new_keys, 'location', 'mountain')
            
        elif location == 'mountain':
            new_keys = msd.taper_face(mesh, fk, height=pnoise1(fk, 2)*10, ratio=perlin.randint(0,1))
            mesh.set_faces_attribute(new_keys, 'location', 'peaks')
            
        elif location == 'sea':
            new_keys = msd.taper_face(mesh, fk, height=pnoise1(fk, 2)*-10, ratio=perlin.randint(0,1))
            mesh.set_faces_attribute(new_keys, 'location', 'sea_grave')
             
        
        elif location == 'sea_grave':
            move_vertices(mesh, fk, -8)
            
            mesh.set_face_attribute(fk, 'location', 'sea_bottom')

        # SIDES
        elif location == 'sides':
            pass
        
        else:
            pass
        
for _ in range(12):
    subdivide_by_ftype(mesh)

In [10]:
draw_compas_mesh(mesh, size=35)

export_obj_by_attribute('inner.obj', mesh, 'location')

In [ ]:
radius = 5
origin = (0., 0., 0.)
count = 0
points = []

# generate 200 random points inside a sphere with given radius
while count < 200:
    x = (random.random() - 0.5) * radius * 2
    y = (random.random() - 0.5) * radius * 2
    z = (random.random() - 0.5) * radius * 2
    pt = x, y, z
    if distance_point_point(origin, pt) <= radius:
        points.append(pt)
        count += 1

# calculate the faces that define the convex hull
faces =  convex_hull(points)

# create a mesh from these
mesh2 = Mesh.from_vertices_and_faces(points, faces)

# remove unreferenced vertices (inside of hull)
vkeys = list(mesh2.vertices())
for v in vkeys:
    if mesh2.vertex_degree(v)==0:
        del mesh2.vertex[v]
        
mesh3 = sd.mesh_subdivide_catmullclark(mesh2)
        
draw_compas_mesh(mesh3)
mesh3.to_obj('sun.obj')